In [ ]:
import visualstimulation as vs
import expipe

In [ ]:
project_path = "/home/alessiob/Documents/Codes/expipe/projects/testing"

project = expipe.get_project(project_path)
actions = project.actions

In [ ]:
actions

In [ ]:
action = actions['1825-300119-1']

In [ ]:
data_path = ps.get_data_path(action)

In [ ]:
epochs = ps.load_epochs(data_path)

In [ ]:
channel_idx = 0
sptr = ps.load_spiketrains(data_path, channel_idx)

In [ ]:
[print(ep.annotations) for ep in epochs]

In [ ]:
oe_epoch = epochs[0]
ps_epoch = epochs[1]

In [ ]:
orients = ps_epoch.labels
print(orients)

In [ ]:
# make trials 
trials = vs.make_spiketrain_trials(sptr[1], oe_epoch)

In [ ]:
vs.add_orientation_to_trials(trials, orients)
print(trials[0].annotations)

In [ ]:
fig = vs.orient_raster_plots(trials)

In [ ]:
fig = vs.plot_tuning_overview(trials)

In [1]:
import visualstimulation as vs
import os

In [2]:
project = os.path.realpath("C:\\apps\\expipe-project\\MROS_V2")
action_id = "1823-060219-2"
o = vs.represent.visual_data(project, action_id)

C:\apps\expipe-project\MROS_V2
actions\1823-060219-2\data\main.exdir


AttributeError: 'visual_data' object has no attribute 'outpath'

In [ ]:
o_sptr = vs.load_spiketrains(str(o.data_path), 0)

In [ ]:
o.get_plot()